# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_data)

extension : .pickle
Loaded 9150 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

1,content,morphs
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반, 에서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 가, 반, 에서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 에서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 에서, 내, 가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]"


In [4]:
df_ref = fm.load(config.temp_ref_data)

extension : .pickle
Loaded 180940 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_ref.pickle


In [5]:
df_ref.cpr.fillna('', inplace=True)
df_ref.cpo.fillna('', inplace=True)

In [6]:
df_ref.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,


# 

# Model Load

In [7]:
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec


In [8]:
okt = Okt()

In [9]:
model = Doc2Vec.load(config.doc2vec)

# 

# Memebot with doc2vec Similarity

### 학습된 문장 유사도

In [10]:
index = 0
print(df.morphs.iloc[index])
print("-"*100)
result = model.docvecs.most_similar(index)
for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.content.iloc[int(v[0])])

['나', '중간고사', '반', '에서', '1등', '했어']
----------------------------------------------------------------------------------------------------
k : 0
v : ('3552', 0.8180486559867859)
나 반에서 인기 투표 1위했어


/var/folders/yv/ss_gxzgs3jx6dn00dwvzvn100000gn/T/ipykernel_10984/749687253.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result = model.docvecs.most_similar(index)


### 

### 입력 문장 유사도

In [33]:
input_sent = input()

기분 좋다


In [34]:
# vectorize new sentences based on doc2vec
input_vec = model.infer_vector(okt.morphs(input_sent))

# most similarity top 5
return_docs = model.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.content.iloc[int(i[0])] for i in return_docs]

# find Similar Sentences in Dialogueset
temp_ref = df_ref[df_ref.u == result[0]]

# result
if temp_ref.empty:
    print("유사 문장이 없습니다.")
else:
    temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
    print(f"입력 문장 : {input_sent}")
    print(f"유사 문장 : {result[0]}")
    print(f"유사도 : {return_docs[0][1]}")
    print("-"*100)
    result = str(temp_ref.cpr.iloc[0]) + " " + str(temp_ref.mm.iloc[0]) + " " + str(temp_ref.cpo.iloc[0])
    print(f"밈 답변 : {result.strip()}")

입력 문장 : 기분 좋다
유사 문장 : 버스는 환기시킬 수 있어서 좋다~
유사도 : 0.5182848572731018
----------------------------------------------------------------------------------------------------
밈 답변 : 인정? 어 인정! 맑은 공기 댕꿀


/var/folders/yv/ss_gxzgs3jx6dn00dwvzvn100000gn/T/ipykernel_10984/1023421750.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  return_docs = model.docvecs.most_similar(positive=[input_vec], topn=5)
